In [1]:
from Parameters import TrainingParameters, EnviromentParameters
from Data import XESDataset
from Models import BaselineLSTM
import tensorflow as tf

%load_ext tensorboard
%tensorboard --logdir logs/gradient_tape --host localhost --port 8088

NameError: name 'torch' is not defined

In [2]:
import tensorflow as tf


<tf.Tensor: shape=(0,), dtype=float32, numpy=array([], dtype=float32)>

In [5]:
a = tf.constant([5,6,7])

In [8]:
tf.expand_dims(a, axis=1)

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[5],
       [6],
       [7]], dtype=int32)>

In [33]:
train_param = TrainingParameters()
xes_dataset = XESDataset(file_path=EnviromentParameters.BPI2020Dataset.file_path,preprocessed_folder_path=EnviromentParameters.BPI2020Dataset.preprocessed_foldr_path,
preprocessed_df_type= EnviromentParameters.BPI2020Dataset.preprocessed_df_type,
include_types = train_param.bpi2012.BPI2012_include_types
)


| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012/OW 


In [34]:
model = BaselineLSTM(
    xes_dataset.vocab, 
    embedding_dim= train_param.baselineLSTMModelParameters.embedding_dim,
    lstm_hidden= train_param.baselineLSTMModelParameters.lstm_hidden,
    dropout=train_param.baselineLSTMModelParameters.dropout
)

In [35]:
optim = tf.keras.optimizers.Adam(learning_rate=0.01)

In [36]:
idx_ds = xes_dataset.get_index_ds()
idx_ds = idx_ds.shuffle(20).batch(train_param.batch_size)

In [37]:
import datetime

In [38]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
tf.keras.callbacks.TensorBoard(log_dir=train_log_dir)

In [39]:
loss_all

<tf.Tensor: shape=(58, 47), dtype=float32, numpy=
array([[5.9724164e-01, 2.8606137e-04, 3.6052385e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [7.7739561e-01, 2.2451689e-03, 8.6974281e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [8.5585576e-01, 6.6609965e-03, 7.9859173e-01, ..., 9.1100201e-02,
        1.6757600e-01, 2.6097195e+00],
       ...,
       [7.8636539e-01, 7.4311293e-04, 3.9882645e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [6.5044618e-01, 1.7907489e-02, 8.4189022e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [6.9888955e-01, 5.6584558e-04, 5.6032395e-01, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)>

In [27]:
import seaborn

In [28]:
import tensorflow as tf

In [29]:
t = tf.Variable(5)

In [50]:
padded_data_traces

array([[ 2, 11, 12, ...,  0,  0,  0],
       [ 2, 17, 18, ...,  0,  0,  0],
       [ 2, 11, 12, ...,  0,  0,  0],
       ...,
       [ 2, 17, 18, ...,  0,  0,  0],
       [ 2, 11, 12, ...,  0,  0,  0],
       [ 2, 17, 18, ...,  0,  0,  0]], dtype=int32)

In [58]:
  pred[0,0,:]

<tf.Tensor: shape=(29,), dtype=float32, numpy=
array([3.8546009e-06, 1.3197672e-03, 1.8784864e-05, 4.4387449e-05,
       1.7493639e-04, 7.5597239e-07, 7.2465150e-06, 5.0433020e-05,
       1.3260893e-05, 1.2333487e-05, 1.0415758e-04, 4.2112643e-01,
       3.6087638e-04, 8.9715577e-06, 3.7928021e-03, 2.7775462e-04,
       2.7722968e-05, 5.7217091e-01, 1.3932257e-04, 9.1593138e-05,
       3.8488903e-05, 8.3363017e-07, 1.5601698e-05, 1.2612489e-05,
       1.0225927e-04, 2.0755089e-05, 4.8622023e-05, 8.3988607e-06,
       6.1023575e-06], dtype=float32)>

In [60]:
import numpy as np

In [78]:
tf.random.categorical(pred, 1).shape

TensorShape([128, 1])

In [65]:
pred.shape

TensorShape([128, 120, 29])

In [66]:
b_size = pred.shape[0]

In [76]:
pred.shape

TensorShape([128, 29])

In [74]:
pred = pred[:, 0, :]

In [44]:
train_param.batch_size

128

In [45]:
steps = 0
for idxs in idx_ds:
    caseids, padded_data_traces, padded_target_traces =  xes_dataset.collate_fn(idxs)
    with tf.GradientTape() as tape:
        pred, _ = model(padded_data_traces, training=True)
        loss_all = tf.keras.losses.sparse_categorical_crossentropy(y_true=padded_target_traces, y_pred=pred)
        loss_all = loss_all * tf.cast(padded_target_traces != 0, dtype=tf.float32)
        loss = tf.reduce_mean(loss_all)
        pred_value = tf.math.argmax(pred, axis=-1)
        accuracy = tf.math.reduce_mean(tf.cast(tf.boolean_mask(padded_target_traces == pred_value,padded_target_traces != 0), dtype= tf.float32)).numpy()
        with train_summary_writer.as_default():
            tf.summary.scalar('accuracy', accuracy, step=steps)
            tf.summary.scalar('loss', loss.numpy(), step=steps)
        print("Acc: %.4f" % (accuracy))
    grads = tape.gradient(loss, model.trainable_variables)
    optim.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
    steps += 1
    raise StopIteration


Acc: 0.8584


StopIteration: 

In [9]:
accuracy

0.5699615

In [10]:
padded_target_traces != 0

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])

In [11]:
acc_m.update_state(padded_target_traces, pred)

NameError: name 'acc_m' is not defined

In [10]:
padded_target_traces

array([[17, 18,  7, ...,  0,  0,  0],
       [11, 12, 10, ...,  0,  0,  0],
       [11, 12, 10, ...,  0,  0,  0],
       ...,
       [11, 12, 10, ...,  0,  0,  0],
       [11, 12, 17, ...,  0,  0,  0],
       [11, 12, 10, ...,  0,  0,  0]], dtype=int32)

In [11]:
pred_value = tf.math.argmax(pred, axis=-1)

0.59887004

In [12]:
acc_m.result().numpy()

0.85876745

In [13]:
tf.math.reduce_mean(tf.cast(padded_target_traces == pred_value, dtype=tf.float32))

<tf.Tensor: shape=(), dtype=float32, numpy=0.85876745>

<tf.Tensor: shape=(58, 47), dtype=float32, numpy=
array([[0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       [0., 1., 0., ..., 1., 1., 1.],
       ...,
       [0., 1., 1., ..., 1., 1., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       [1., 1., 0., ..., 1., 1., 1.]], dtype=float32)>